In [1]:
!git clone https://github.com/mcordts/cityscapesScripts.git

Cloning into 'cityscapesScripts'...
remote: Enumerating objects: 640, done.
remote: Counting objects: 100% (213/213), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 640 (delta 179), reused 163 (delta 156), pack-reused 427
Receiving objects: 100% (640/640), 793.19 KiB | 1.01 MiB/s, done.
Resolving deltas: 100% (365/365), done.


In [2]:
!cd cityscapesScripts && python3 setup.py install

running install
running bdist_egg
running egg_info
creating cityscapesScripts.egg-info
writing cityscapesScripts.egg-info/PKG-INFO
writing dependency_links to cityscapesScripts.egg-info/dependency_links.txt
writing entry points to cityscapesScripts.egg-info/entry_points.txt
writing requirements to cityscapesScripts.egg-info/requires.txt
writing top-level names to cityscapesScripts.egg-info/top_level.txt
writing manifest file 'cityscapesScripts.egg-info/SOURCES.txt'
reading manifest file 'cityscapesScripts.egg-info/SOURCES.txt'
writing manifest file 'cityscapesScripts.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/cityscapesscripts
copying cityscapesscripts/__init__.py -> build/lib/cityscapesscripts
creating build/lib/cityscapesscripts/download
copying cityscapesscripts/download/__init__.py -> build/lib/cityscapesscripts/download
copying cityscapesscripts/download/dow

In [ ]:
!python3 cityscapesScripts/cityscapesscripts/download/downloader.py leftImg8bit_trainvaltest.zip
!python3 cityscapesScripts/cityscapesscripts/download/downloader.py gtFine_trainvaltest.zip


In [ ]:
!unzip leftImg8bit_trainvaltest.zip -d /app/duy55/segmentation/cityscapes
!unzip gtFine_trainvaltest.zip -d /app/duy55/segmentation/cityscapes

### The code to be used on python project is from this notebook ###

In [1]:
import math
import random
import numbers
import numpy as np

from PIL import Image, ImageOps


def resize(img_w, img_h, size, max_size=1000):
    """
    Resize the input PIL image to the given size.
    The bounding boxes change the same.
    :param img_h:
    :param img_w:
    :param size:     (tuple or int)
                     - if is tuple, resize image to the size.
                     - if is int, resize the shorter side to the size while maintaining the aspect ratio.
    :param max_size: (int) when size is int, limit the image longer size to max_size.
                     - this is essential to limit the usage of GPU memory.
    :return:

    """
    if isinstance(size, int):
        size_min = min(img_w, img_h)
        size_max = max(img_w, img_h)

        sw = sh = float(size) / size_min
        if sw * size_max > max_size:
            sw = sh = float(max_size) / size_max

        ow = int(img_w * sw + 0.5)
        oh = int(img_h * sh + 0.5)
    else:
        ow, oh = size
        sw = float(ow) / img_w
        sh = float(oh) / img_h
    return ow, oh, sw, sh


def random_crop(img_w, img_h):

    ox, oy, ow, oh = 0, 0, img_w, img_h
    success = False
    for attempt in range(10):
        area = img_w * img_h
        target_area = random.uniform(0.56, 1.0) * area
        aspect_ratio = random.uniform(3. / 4, 4. / 3)

        ow = int(round(math.sqrt(target_area * aspect_ratio)))
        oh = int(round(math.sqrt(target_area / aspect_ratio)))

        if random.random() < 0.5:
            ow, oh = oh, ow

        if ow <= img_w and oh <= img_h:
            ox = random.randint(0, img_w - ow)
            oy = random.randint(0, img_h - oh)
            success = True
            break

    # Fallback
    if not success:
        ow = oh = min(img_w, img_h)
        ox = (img_w - ow) // 2
        oy = (img_h - oh) // 2

    return ox, oy, ow, oh


def center_crop(img_w, img_h, size):

    ow, oh = size
    ox = int(round((img_w - ow) / 2.))
    oy = int(round((img_h - oh) / 2.))

    return ox, oy, ow, oh


class Compose(object):
    def __init__(self, augmentations):
        self.augmentations = augmentations

    def __call__(self, img, mask):
        img, mask = Image.fromarray(img, mode='RGB'), Image.fromarray(mask, mode='L')
        assert img.size == mask.size

        for a in self.augmentations:
            img, mask = a(img, mask)
        return np.array(img, dtype=np.uint8), np.array(mask, dtype=np.uint8)


class RandomCrop(object):
    def __init__(self, size, padding=0):
        if isinstance(size, numbers.Number):
            self.size = (int(size), int(size))
        else:
            self.size = size
        self.padding = padding

    def __call__(self, img, mask):
        if self.padding > 0:
            img = ImageOps.expand(img, border=self.padding, fill=0)
            mask = ImageOps.expand(mask, border=self.padding, fill=0)

        assert img.size == mask.size

        w, h = img.size
        th, tw = self.size
        if w == tw and h == th:
            return img, mask
        if w < tw or h < th:
            return img.resize((tw, th), Image.BILINEAR), mask.resize((tw, th), Image.NEAREST)

        x1 = random.randint(0, w - tw)
        y1 = random.randint(0, h - th)
        return img.crop((x1, y1, x1 + tw, y1 + th)), mask.crop((x1, y1, x1 + tw, y1 + th))


class CenterCrop(object):
    def __init__(self, size):
        if isinstance(size, numbers.Number):
            self.size = (int(size), int(size))
        else:
            self.size = size

    def __call__(self, img, mask):
        assert img.size == mask.size
        w, h = img.size
        th, tw = self.size
        x1 = int(round((w - tw) / 2.))
        y1 = int(round((h - th) / 2.))
        return img.crop((x1, y1, x1 + tw, y1 + th)), mask.crop((x1, y1, x1 + tw, y1 + th))


class RandomHorizontallyFlip(object):
    def __call__(self, img, mask):
        if random.random() < 0.5:
            return img.transpose(Image.FLIP_LEFT_RIGHT), mask.transpose(Image.FLIP_LEFT_RIGHT)
        return img, mask


class FreeScale(object):
    def __init__(self, size):
        self.size = tuple(reversed(size))  # size: (h, w)

    def __call__(self, img, mask):
        assert img.size == mask.size
        return img.resize(self.size, Image.BILINEAR), mask.resize(self.size, Image.NEAREST)


class Scale(object):
    def __init__(self, size):
        self.size = size

    def __call__(self, img, mask):
        assert img.size == mask.size
        w, h = img.size
        if (w >= h and w == self.size[1]) or (h >= w and h == self.size[0]):
            return img, mask

        oh, ow = self.size
        return img.resize((ow, oh), Image.BILINEAR), mask.resize((ow, oh), Image.NEAREST)


class RandomSizedCrop(object):
    def __init__(self, size):
        self.size = size

    def __call__(self, img, mask):
        assert img.size == mask.size
        for attempt in range(10):
            area = img.size[0] * img.size[1]
            target_area = random.uniform(0.45, 1.0) * area
            aspect_ratio = random.uniform(0.5, 2)

            w = int(round(math.sqrt(target_area * aspect_ratio)))
            h = int(round(math.sqrt(target_area / aspect_ratio)))

            if random.random() < 0.5:
                w, h = h, w

            if w <= img.size[0] and h <= img.size[1]:
                x1 = random.randint(0, img.size[0] - w)
                y1 = random.randint(0, img.size[1] - h)

                img = img.crop((x1, y1, x1 + w, y1 + h))
                mask = mask.crop((x1, y1, x1 + w, y1 + h))
                assert (img.size == (w, h))

                return img.resize((self.size, self.size), Image.BILINEAR), mask.resize((self.size, self.size),
                                                                                       Image.NEAREST)

        # Fallback
        scale = Scale(self.size)
        crop = CenterCrop(self.size)
        return crop(*scale(img, mask))


class RandomRotate(object):
    def __init__(self, degree):
        self.degree = degree

    def __call__(self, img, mask):
        rotate_degree = random.random() * 2 * self.degree - self.degree
        return img.rotate(rotate_degree, Image.BILINEAR), mask.rotate(rotate_degree, Image.NEAREST)


class RandomSized(object):
    def __init__(self, limit):
        self.limit = limit

    def __call__(self, img, mask):

        scale = random.uniform(self.limit[0], self.limit[1])
        w = int(scale * img.size[0])
        h = int(scale * img.size[1])

        img, mask = img.resize((w, h), Image.BILINEAR), mask.resize((w, h), Image.NEAREST)

        return img, mask

In [2]:
def recursive_glob(rootdir='.', suffix=''):
    """Performs recursive glob with given suffix and rootdir
        :param rootdir is the root directory
        :param suffix is the suffix to be searched
    """
    return [os.path.join(looproot, filename)
            for looproot, _, filenames in os.walk(rootdir)
            for filename in filenames if filename.endswith(suffix)]

In [3]:
import os
import torch
import imageio as misc

from torch.utils import data
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch.nn as nn
import sklearn.metrics as skm
import torch.optim as optim
from tqdm import tqdm
import matplotlib.pyplot as plt  
import torch.nn.functional as F
import time

class CityscapesLoader(Dataset):
    """
    CityscapesLoader

    https://www.cityscapes-dataset.com

    Data is derived from CityScapes, and can be downloaded from here:
    https://www.cityscapes-dataset.com/downloads/

    Many Thanks to @fvisin for the loader repo:
    https://github.com/fvisin/dataset_loaders/blob/master/dataset_loaders/images/cityscapes.py
    """
    colors = [  # [  0,   0,   0],
        [128, 64, 128],
        [244, 35, 232],
        [70, 70, 70],
        [102, 102, 156],
        [190, 153, 153],
        [153, 153, 153],
        [250, 170, 30],
        [220, 220, 0],
        [107, 142, 35],
        [152, 251, 152],
        [0, 130, 180],
        [220, 20, 60],
        [255, 0, 0],
        [0, 0, 142],
        [0, 0, 70],
        [0, 60, 100],
        [0, 80, 100],
        [0, 0, 230],
        [119, 11, 32]]

    label_colours = dict(zip(range(19), colors))

    def __init__(self, root, split="train", gt="gtFine", img_size=(512, 1024),
                 is_transform=False, augmentations=None):
        """
        :param root:         (str)  Path to the datasets sets root
        :param split:        (str)  Data set split -- 'train' 'train_extra' or 'val'
        :param gt:           (str)  Type of ground truth label -- 'gtFine' or 'gtCoarse'
        :param img_size:     (tuple or int) The size of the input image
        :param is_transform: (bool) Transform the image or not
        :param augmentations (object) Data augmentations used in the image and label
        """
        self.root = root
        self.gt = gt
        self.split = split
        self.is_transform = is_transform
        self.augmentations = augmentations

        self.n_classes = 19
        self.img_size = img_size if isinstance(img_size, tuple) else (img_size, img_size)
        self.mean = np.array([73.16, 82.91, 72.39])
        self.files = {}

        self.images_base = os.path.join(self.root, 'leftImg8bit', self.split)
        self.annotations_base = os.path.join(self.root, gt, self.split)

        self.files[split] = recursive_glob(rootdir=self.images_base, suffix='.png')

        self.void_classes = [0, 1, 2, 3, 4, 5, 6, 9, 10, 14, 15, 16, 18, 29, 30, -1]
        self.valid_classes = [7, 8, 11, 12, 13, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 31, 32, 33]
        self.class_names = ['unlabelled', 'road', 'sidewalk', 'building', 'wall', 'fence',
                            'pole', 'traffic_light', 'traffic_sign', 'vegetation', 'terrain',
                            'sky', 'person', 'rider', 'car', 'truck', 'bus', 'train',
                            'motorcycle', 'bicycle']

        self.ignore_index = 250
        self.class_map = dict(zip(self.valid_classes, range(19)))

        if not self.files[split]:
            raise Exception("> No files for split=[%s] found in %s" % (split, self.images_base))

        print("> Found %d %s images..." % (len(self.files[split]), split))

    def __len__(self):
        """__len__"""
        return len(self.files[self.split])

    def __getitem__(self, index):
        """__getitem__

        :param index:
        """
        img_path = self.files[self.split][index].rstrip()
        lbl_path = os.path.join(self.annotations_base,
                                img_path.split(os.sep)[-2],
                                os.path.basename(img_path)[:-15] + '{}_labelIds.png'.format(self.gt))

        if not os.path.isfile(img_path) or not os.path.exists(img_path):
            raise Exception("{} is not a file, can not open with imread.".format(img_path))

        img = misc.imread(img_path)
        img = np.array(img, dtype=np.uint8)
        # img = misc.imresize(img, (self.img_size[0], self.img_size[1], "bilinear"))

        if not os.path.isfile(lbl_path) or not os.path.exists(lbl_path):
            raise Exception("{} is not a file, can not open with imread.".format(lbl_path))

        lbl = misc.imread(lbl_path)
        # lbl = misc.imresize(lbl, (self.img_size[0], self.img_size[1]), "nearest", mode='F')
        lbl = self.encode_segmap(np.array(lbl, dtype=np.uint8))

        if self.augmentations is not None:
            img, lbl = self.augmentations(img, lbl)

        if self.is_transform:
            img, lbl = self.transform(img, lbl)

        img = torch.from_numpy(img).float()
        lbl = torch.from_numpy(lbl).long()
        return img, lbl

    def transform(self, img, lbl):
        """transform

        :param img:
        :param lbl:
        """
        img = img[:, :, ::-1]         # From RGB to BGR
        img = img.astype(float)
        img -= self.mean
        img /= 255.0
        img = img.transpose(2, 0, 1)  # From H*W*C to C*H*W

        if not np.all(np.unique(lbl[lbl != self.ignore_index]) < self.n_classes):
            raise ValueError("> Segmentation map contained invalid class values.")

        return img, lbl

    def decode_segmap(self, temp):
        r = temp.copy()
        g = temp.copy()
        b = temp.copy()
        for l in range(0, self.n_classes):
            r[temp == l] = self.label_colours[l][0]
            g[temp == l] = self.label_colours[l][1]
            b[temp == l] = self.label_colours[l][2]

        rgb = np.zeros((temp.shape[0], temp.shape[1], 3))
        rgb[:, :, 0] = r / 255.0
        rgb[:, :, 1] = g / 255.0
        rgb[:, :, 2] = b / 255.0
        return rgb

    def encode_segmap(self, mask):
        # Put all void classes to zero
        for _voidc in self.void_classes:
            mask[mask == _voidc] = self.ignore_index
        for _validc in self.valid_classes:
            mask[mask == _validc] = self.class_map[_validc]
        return mask


# +++++++++++++++++++++++++++++++++++++++++++++ #
# Test the code of 'CityscapesLoader'
# +++++++++++++++++++++++++++++++++++++++++++++ #


In [4]:
net_h, net_w = 448, 448
augment = Compose([RandomHorizontallyFlip(), RandomSized((0.625, 0.75)),
                       RandomRotate(6), RandomCrop((net_h, net_w))])

local_path = "/app/duy55/segmentation/cityscapes"
train_data = CityscapesLoader(local_path, split="train", is_transform=True, augmentations=augment)
val_data = CityscapesLoader(local_path, split="val", is_transform=False, augmentations=None)

> Found 2975 train images...
> Found 500 val images...


In [5]:
# replace device accordingly
device = torch.device('cuda:0')

# replace with location of folder containing "gtFine" and "leftImg8bit"
path_data = "/app/duy55/segmentation/cityscapes"

learning_rate = 1e-4
train_epochs = 8
n_classes = 19
batch_size = 2
num_workers = 0

In [6]:

train_loader = DataLoader(
    train_data,
    batch_size = batch_size,
    shuffle=True,
    num_workers = num_workers,
    #pin_memory = pin_memory  # gave no significant advantage
)

val_loader = DataLoader(
    val_data,
    batch_size = batch_size,
    num_workers = num_workers,
    #pin_memory = pin_memory  # gave no significant advantage
)

In [11]:
##############################################################################################################################
##### My FCN #################################################################################################################

def down_conv(small_channels, big_channels, pad):   ### contracting block
    return torch.nn.Sequential(
        torch.nn.Conv2d(small_channels, big_channels, 3, padding=pad),
        torch.nn.ReLU(),
        torch.nn.BatchNorm2d(big_channels),
        torch.nn.Conv2d(big_channels, big_channels, 3, padding=pad),
        torch.nn.ReLU(),
        torch.nn.BatchNorm2d(big_channels)
    )   ## consider stride = 2

def up_conv(big_channels, small_channels, pad):
    return torch.nn.Sequential(
        torch.nn.Conv2d(big_channels, small_channels, 3, padding=pad),
        torch.nn.ReLU(),
        torch.nn.BatchNorm2d(small_channels),
        torch.nn.Conv2d(small_channels, small_channels, 3, padding=pad),
        torch.nn.ReLU(),
        torch.nn.BatchNorm2d(small_channels)
    )


class my_FCN(torch.nn.Module):
    def crop(self, a, b):
        ## a, b tensor shape = [batch, channel, H, W]
        Ha = a.size()[2]
        Wa = a.size()[3]
        Hb = b.size()[2]
        Wb = b.size()[3]

        adapt = torch.nn.AdaptiveMaxPool2d((Ha,Wa))
        crop_b = adapt(b) 
            
        return crop_b    
   
    
    def __init__(self, num_classes):
        super().__init__()
        self.num_classes = num_classes
        self.relu    = torch.nn.ReLU()
        self.maxpool = torch.nn.MaxPool2d(kernel_size=2, ceil_mode=True)         
        self.mean = torch.Tensor([0.5, 0.5, 0.5])
        self.std = torch.Tensor([0.25, 0.25, 0.25])
        
        a = 32
        b = a*2 #64
        c = b*2 #128
        d = c*2 #256
        
        n_class = self.num_classes
        
        self.conv_down1 = down_conv(3, a, 1) # 3 --> 32
        self.conv_down2 = down_conv(a, b, 1)  # 32 --> 64
        self.conv_down3 = down_conv(b, c, 1)  # 64 --> 128
        self.conv_down4 = down_conv(c, d, 1)  # 128 --> 256
        
        self.bottleneck = torch.nn.ConvTranspose2d(d, c, kernel_size=3, stride=2, padding=1, output_padding=1)  
        self.conv_up3 = up_conv(c, b, 1)  # 128 --> 64
        self.upsample3 = torch.nn.ConvTranspose2d(b, a, kernel_size=3, stride=2, padding=1, output_padding=1)   
                 
        self.classifier = torch.nn.Conv2d(a, n_class, kernel_size=1) 
        
    
    def forward(self, x):
        H = x.shape[2]
        W = x.shape[3]
        z = (x - self.mean[None, :, None, None].to(x.device)) / self.std[None, :, None, None].to(x.device)
        #################### DOWN / ENCODER #############################
        conv1 =  self.conv_down1(z)   # 3 --> 32
        mx1 = self.maxpool(conv1)
        conv2 =  self.conv_down2(mx1)  # 32 --> 64
        mx2 = self.maxpool(conv2) 
        conv3 =  self.conv_down3(mx2) # 64 --> 128  
        mx3 = self.maxpool(conv3) 
        conv4 =  self.conv_down4(conv3) # 128 --> 256  ################### CHANGED THIS

        ########################### BOTTLENECK #############################
        score = self.bottleneck(conv4)  # 256 --> 128
       
        ######################### UP/DECODER #######################
        crop_conv3 = self.crop(score, conv3)    
        score = score + crop_conv3   ### add 128 
        
        ##########################
        score = self.conv_up3(score)  # 128 --> 64
        score = self.upsample3(score)  # 64 --> 32     
        crop_conv1 = self.crop(score, conv1)   
        score = score + crop_conv1   ### add 32           
        
        ############################
        score = self.classifier(score) 
        out = torch.nn.functional.interpolate(score, size=(H,W))
        out = out[:, :, :H, :W]
        return out  


In [12]:
# Instance of the model defined above.
model = my_FCN(n_classes).to(device)

In [13]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Cross Entropy Loss adapted from meetshah1995 to prevent size inconsistencies between model precition 
# and target label
# https://github.com/meetshah1995/pytorch-semseg/blob/master/ptsemseg/loss/loss.py

def cross_entropy2d(input, target, weight=None):
    n, c, h, w = input.size()
    nt, ht, wt = target.size()

    # Handle inconsistent size between input and target
    if h != ht and w != wt:  # upsample labels
        input = F.interpolate(input, size=(ht, wt), mode="bilinear", align_corners=True)

    input = input.transpose(1, 2).transpose(2, 3).contiguous().view(-1, c)
    target = target.view(-1)
    loss = F.cross_entropy(
        input, target, weight=weight, reduction="mean", ignore_index=250
    )
    return loss

In [14]:
'''We have used skelarn libraries to calculate Accuracy and Jaccard Score'''

def get_metrics(gt_label, pred_label):
    #Accuracy Score
    acc = skm.accuracy_score(gt_label, pred_label, normalize=True)
    
    #Jaccard Score/IoU
    js = skm.jaccard_score(gt_label, pred_label, average='micro')
    
    result_gm_sh = [acc, js]
    return(result_gm_sh)

class runningScore(object):
    def __init__(self, n_classes):
        self.n_classes = n_classes
        self.confusion_matrix = np.zeros((n_classes, n_classes))

    def _fast_hist(self, label_true, label_pred, n_class):
        mask = (label_true >= 0) & (label_true < n_class)
        hist = np.bincount(
            n_class * label_true[mask].astype(int) + label_pred[mask], minlength=n_class ** 2
        ).reshape(n_class, n_class)
        return hist

    def update(self, label_trues, label_preds):
        for lt, lp in zip(label_trues, label_preds):
            self.confusion_matrix += self._fast_hist(lt.flatten(), lp.flatten(), self.n_classes)

    def get_scores(self):
        # confusion matrix
        hist = self.confusion_matrix
        
        #              T
        #         0    1    2
        #    0   TP   FP   FP
        #  P 1   FN   TN   TN       This is wrt to class 0
        #    2   FN   TN   TN

        #         0    1    2
        #    0   TP   FP   FP
        #  P 1   FP   TP   FP       This is wrt prediction classes; AXIS = 1
        #    2   FP   FP   TP 

        #         0    1    2
        #    0   TP   FN   FN
        #  P 1   FN   TP   FN       This is wrt true classes; AXIS = 0
        #    2   FN   FN   TP   

        TP = np.diag(hist)
        TN = hist.sum() - hist.sum(axis = 1) - hist.sum(axis = 0) + np.diag(hist)
        FP = hist.sum(axis = 1) - TP
        FN = hist.sum(axis = 0) - TP
        
        # 1e-6 was added to prevent corner cases where denominator = 0
        
        # Specificity: TN / TN + FP
        specif_cls = (TN) / (TN + FP + 1e-6)
        specif = np.nanmean(specif_cls)
        
        # Senstivity/Recall: TP / TP + FN
        sensti_cls = (TP) / (TP + FN + 1e-6)
        sensti = np.nanmean(sensti_cls)
        
        # Precision: TP / (TP + FP)
        prec_cls = (TP) / (TP + FP + 1e-6)
        prec = np.nanmean(prec_cls)
        
        # F1 = 2 * Precision * Recall / Precision + Recall
        f1 = (2 * prec * sensti) / (prec + sensti + 1e-6)
        
        return (
            {
                "Specificity": specif,
                "Senstivity": sensti,
                "F1": f1,
            }
        )

    def reset(self):
        self.confusion_matrix = np.zeros((self.n_classes, self.n_classes))

In [15]:
def train(train_loader, model, optimizer, epoch_i, epoch_total):
        count = 0
        
        # List to cumulate loss during iterations
        loss_list = []
        for (images, labels) in train_loader:
            count += 1
            
            # we used model.eval() below. This is to bring model back to training mood.
            model.train()

            images = images.to(device)
            labels = labels.to(device)
            
            # Model Prediction
            pred = model(images)
            
            # Loss Calculation
            loss = cross_entropy2d(pred, labels)
            loss_list.append(loss)

            # optimiser
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            # interval to print train statistics
            if count % 50 == 0:
                fmt_str = "Image: {:d} in epoch: [{:d}/{:d}]  and Loss: {:.4f}"
                print_str = fmt_str.format(
                    count,
                    epoch_i + 1,
                    epoch_total,
                    loss.item()
                )
                print(print_str)
                   
#           # break for testing purpose
#             if count == 10:
#                 break
        return(loss_list)

In [16]:
def validate(val_loader, model, epoch_i):
    
    # tldr: to make layers behave differently during inference (vs training)
    model.eval()
    
    # enable calculation of confusion matrix for n_classes = 19
    running_metrics_val = runningScore(19)
    
    # empty list to add Accuracy and Jaccard Score Calculations
    acc_sh = []
    js_sh = []
    
    with torch.no_grad():
        for image_num, (val_images, val_labels) in tqdm(enumerate(val_loader)):
            
            val_images = val_images.to(device)
            val_labels = val_labels.to(device)
            
            # Model prediction
            val_pred = model(val_images)
            
            # Coverting val_pred from (1, 19, 512, 1024) to (1, 512, 1024)
            # considering predictions with highest scores for each pixel among 19 classes
            pred = val_pred.data.max(1)[1].cpu().numpy()
            gt = val_labels.data.cpu().numpy()
            
            # Updating Mertics
            running_metrics_val.update(gt, pred)
            sh_metrics = get_metrics(gt.flatten(), pred.flatten())
            acc_sh.append(sh_metrics[0])
            js_sh.append(sh_metrics[1])
                               
#            # break for testing purpose
#             if image_num == 10:
#                 break                

    score = running_metrics_val.get_scores()
    running_metrics_val.reset()
    
    acc_s = sum(acc_sh)/len(acc_sh)
    js_s = sum(js_sh)/len(js_sh)
    score["acc"] = acc_s
    score["js"] = js_s
    
    print("Different Metrics were: ", score)  
    return(score)

In [17]:
if __name__ == "__main__":

    # to hold loss values after each epoch
    loss_all_epochs = []
    
    # to hold different metrics after each epoch
    Specificity_ = []
    Senstivity_ = []
    F1_ = []
    acc_ = []
    js_ = []
    
    for epoch_i in range(train_epochs):
        # training
        print(f"Epoch {epoch_i + 1}\n-------------------------------")
        t1 = time.time()
        loss_i = train(train_loader, model, optimizer, epoch_i, train_epochs)
        loss_all_epochs.append(loss_i)
        t2 = time.time()
        print("It took: ", t2-t1, " unit time")

        # metrics calculation on validation data
        dummy_list = validate(val_loader, model, epoch_i)   
        
        # Add metrics to empty list above
        Specificity_.append(dummy_list["Specificity"])
        Senstivity_.append(dummy_list["Senstivity"])
        F1_.append(dummy_list["F1"])
        acc_.append(dummy_list["acc"])
        js_.append(dummy_list["js"])

Epoch 1
-------------------------------


/tmp/ipykernel_4792/3088237474.py:109: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = misc.imread(img_path)
/tmp/ipykernel_4792/3088237474.py:116: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  lbl = misc.imread(lbl_path)


/usr/local/lib/python3.8/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


Image: 50 in epoch: [1/8]  and Loss: 1.9537
Image: 100 in epoch: [1/8]  and Loss: 2.5519
Image: 150 in epoch: [1/8]  and Loss: 1.8789
Image: 200 in epoch: [1/8]  and Loss: 2.0266
Image: 250 in epoch: [1/8]  and Loss: 1.6694
